In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [4]:
x[0].shape

(1, 28, 28)

In [5]:
x[0, 0]

array([[6.51265254e-01, 9.19869958e-01, 4.28268562e-01, 4.72315828e-01,
        8.10747960e-01, 1.14934213e-01, 7.08445724e-01, 8.25380692e-01,
        1.06981882e-01, 3.37168730e-01, 2.34399384e-01, 2.04877413e-01,
        7.06638892e-01, 1.67315044e-01, 9.91516775e-01, 5.65863093e-01,
        5.58758764e-01, 8.39630724e-01, 4.28043222e-01, 5.49478622e-01,
        7.83312529e-01, 2.54689897e-01, 5.01047143e-01, 3.35017630e-01,
        4.92967310e-02, 8.12326005e-01, 9.95221604e-01, 9.30766274e-01],
       [4.14891364e-01, 1.98553897e-01, 9.94999657e-01, 6.67050918e-01,
        4.08504844e-01, 6.61121232e-01, 4.75505884e-01, 3.36660419e-01,
        5.90102099e-01, 6.85557509e-01, 7.39797169e-01, 8.42508000e-01,
        9.17516650e-01, 3.78576480e-01, 1.93833202e-02, 9.04147756e-01,
        8.79824896e-01, 3.04763870e-02, 5.98374336e-01, 1.15967019e-01,
        4.56393027e-01, 3.58540991e-01, 1.78882114e-01, 9.87300432e-01,
        9.38348271e-01, 1.32659234e-02, 1.06884925e-02, 7.00473

In [7]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        sefl.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 展開(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 最大値(2)
        out = np.max(col, axis=1)
        # 整形(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

In [15]:
# 7.5 CNNの実装
class SimpleConvNet:
    def __init__(
        self,
        input_dim=(1, 28, 28),
        conv_param={'filter_num':30, 'filter_size': 5},
        hidden_size=100,
        output_size=10,
        weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = wight_init_std * np.random.randn(hidden_size, ouput_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_prams['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 設定
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads